# JIRA TICKETING AUTOMATION

## A. API Get Jira Request fields labels, fields responses, 

In [1]:
#Import necessary libraries

import requests
from requests.auth import HTTPBasicAuth
import json
import os
from dotenv import load_dotenv
import psycopg2 #to connect to PostgreSQL database
from psycopg2.extras import execute_values #a function to insert multiple records into the database

In [2]:
#load_dotenv() variables and secrets
load_dotenv()
admin_email = os.getenv("admin_email")  
API_TOKEN =os.getenv("API_TOKEN") 
FIELDVALUE_API=os.getenv("FIELDVALUE_API")
JIRA_URL= os.getenv("JIRA_URL") 
#"https://declaunchpad.atlassian.net" #
REQUESTAPI_TOKEN=os.getenv("REQUESTAPI_TOKEN")
CLOUD_ID= os.getenv("CLOUD_ID") 
#C_ID=os.getenv("C_ID")
REQUESTTYPEID = os.getenv("REQUESTTYPEID") 
SERVICEDESKID = os.getenv("SERVICEDESKID") 
FORMTEMPLATE_ID = os.getenv("FORMTEMPLATE_ID")




### A1. Fetching the fields required to create customer request:

These fields are basically the Summary and Description field called "What are the details of your request?" in the ticket

In [3]:
#Get all the field question from Jira request form

url = f"{JIRA_URL}/rest/servicedeskapi/servicedesk/{SERVICEDESKID}/requesttype/{REQUESTTYPEID}/field"
#url

auth = HTTPBasicAuth(admin_email,REQUESTAPI_TOKEN)
headers = {
"Authorization": "Basic <base64 encoded email:token>",
"Accept": "application/json"
}

response = requests.get(url, headers=headers, auth=auth)

requestform = json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": "))
requestform



'{\n    "canAddRequestParticipants": true,\n    "canRaiseOnBehalfOf": true,\n    "requestTypeFields": [\n        {\n            "defaultValues": [],\n            "description": "",\n            "fieldId": "summary",\n            "jiraSchema": {\n                "system": "summary",\n                "type": "string"\n            },\n            "name": "Summary",\n            "required": true,\n            "validValues": [],\n            "visible": true\n        },\n        {\n            "defaultValues": [],\n            "description": "",\n            "fieldId": "description",\n            "jiraSchema": {\n                "system": "description",\n                "type": "string"\n            },\n            "name": "What are the details of your request?",\n            "required": true,\n            "validValues": [],\n            "visible": true\n        }\n    ]\n}'

### A2. Getting the form fields linked to the request type:
**These fields are called the options fields where you have dropdown to select from.**

**This would return the form structure, the questions ID and the options under the drop down questions.**
**Also, the form template Id would be derived from the output of the script**

In [4]:
# Get all field value in the dropdown type of questions
url = f'https://api.atlassian.com/jira/forms/cloud/{CLOUD_ID}/servicedesk/{SERVICEDESKID}/requesttype/{REQUESTTYPEID}/form'

auth = HTTPBasicAuth(admin_email, FIELDVALUE_API)
#url

headers = {
"Authorization": "Basic <base64 encoded email:token>",
"Accept": "application/json"
}

response = requests.get(url, headers=headers, auth=auth)

formfield = json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": "))
formfield


'{\n    "design": {\n        "conditions": {\n            "207": {\n                "i": {\n                    "co": {\n                        "cIds": {\n                            "205": [\n                                "183"\n                            ]\n                        }\n                    }\n                },\n                "o": {\n                    "sIds": [\n                        "1"\n                    ],\n                    "t": "sh"\n                }\n            }\n        },\n        "layout": [\n            {\n                "content": [\n                    {\n                        "attrs": {\n                            "level": 2,\n                            "localId": "a9fee306-84e4-4468-be21-0692ff382104"\n                        },\n                        "content": [\n                            {\n                                "text": "Person making request",\n                                "type": "text"\n                         

### A3. Fetching manually created customer request responses

**This is an example of a ticket created manually and the response for the requestform which would have the Summary and Description answer (What are the details of your request?).**

In [5]:
# Get all field value in the dropdown type of questions
url = f'{JIRA_URL}/rest/servicedeskapi/request'

auth = HTTPBasicAuth(admin_email, FIELDVALUE_API)
#url

#headers = {
#"Authorization": "Basic <base64 encoded email:token>",
#"Accept": "application/json"
#}

headers = {
  "Accept": "application/json",
  "Content-Type": "application/json",
  "Authorization": "Bearer <access_token>"
}

#data=json.dumps({})

response = requests.get(url, 
                         #data = data
                         headers=headers, auth=auth)

ticket= json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": "))
ticket


'{\n    "_expands": [\n        "participant",\n        "status",\n        "sla",\n        "requestType",\n        "serviceDesk",\n        "attachment",\n        "action",\n        "comment"\n    ],\n    "_links": {\n        "base": "https://declaunchpad.atlassian.net",\n        "context": "",\n        "self": "https://declaunchpad.atlassian.net/rest/servicedeskapi/request"\n    },\n    "isLastPage": true,\n    "limit": 50,\n    "size": 1,\n    "start": 0,\n    "values": [\n        {\n            "_expands": [\n                "participant",\n                "status",\n                "sla",\n                "requestType",\n                "serviceDesk",\n                "attachment",\n                "action",\n                "comment"\n            ],\n            "_links": {\n                "agent": "https://declaunchpad.atlassian.net/browse/CCX-1",\n                "jiraRest": "https://declaunchpad.atlassian.net/rest/api/2/issue/10033",\n                "self": "https://declaunchpa

To get the Form template ID of the Issue, the get issue id API was used

In [6]:
# Get the form template id
url = f'https://api.atlassian.com/jira/forms/cloud/{CLOUD_ID}/issue/CCX-1/form'

auth = HTTPBasicAuth(admin_email, FIELDVALUE_API)
#url

headers = {
  "Accept": "application/json",
  "Content-Type": "application/json",
  "Authorization": "Bearer <access_token>"
}

response = requests.get(url, 
                         #data = data
                         headers=headers, auth=auth)

print(json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": ")))

[
    {
        "formTemplate": {
            "id": "336bb096-c808-465b-a5f1-c4fb2de738d5"
        },
        "id": "5f60ce33-1fdd-424a-8213-049c4f8a6f9a",
        "internal": false,
        "lock": false,
        "name": "Phone equipment order",
        "submitted": true,
        "updated": "2025-11-10T11:01:53.931Z"
    }
]


### API Get Script to get the field responses for a filled request

In [7]:
# Get all field responses in the dropdown type of questions
url = f'https://api.atlassian.com/jira/forms/cloud/{CLOUD_ID}/request/CCX-1/form/5f60ce33-1fdd-424a-8213-049c4f8a6f9a/format/answers'


auth = HTTPBasicAuth(admin_email, FIELDVALUE_API)
#url

headers = {
"Authorization": "Basic <base64 encoded email:token>",
"Accept": "application/json"
}

# headers = {
#   "Accept": "application/json",
#   "Content-Type": "application/json",
#   "Authorization": "Bearer <access_token>"
# }

#data=json.dumps({})

response = requests.get(url, 
                         #data = data
                         headers=headers, auth=auth)

responses = json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": "))
responses

'[\n    {\n        "answer": "adeboladesoyin@gmail.com",\n        "label": " Email"\n    },\n    {\n        "answer": "Thank you.",\n        "label": "Comments"\n    },\n    {\n        "answer": "ADEBOLA ADESOYIN",\n        "label": "Name"\n    },\n    {\n        "answer": "08169567636",\n        "label": "Phone"\n    },\n    {\n        "answer": "IT",\n        "label": "Department"\n    },\n    {\n        "answer": "Data",\n        "label": "Job title"\n    },\n    {\n        "answer": "",\n        "label": "Cost center (acct #)"\n    },\n    {\n        "answer": "2025-11-11",\n        "label": "Date needed by"\n    },\n    {\n        "answer": "Mobile phone",\n        "choice": "167",\n        "label": "Equipment requested"\n    },\n    {\n        "answer": "Relocate existing to a new location ",\n        "choice": "182",\n        "label": "Installation requested"\n    },\n    {\n        "answer": "Permanent use",\n        "choice": "184",\n        "label": "This equipment is for "\n

In [8]:
load_dotenv()
SB_USER = os.getenv("SB_USER")
SB_PASSWORD = os.getenv("SB_PASSWORD")
SB_HOST = os.getenv("SB_HOST")
SB_PORT = os.getenv("SB_PORT")
SB_NAME = os.getenv("SB_NAME")


DB_USER = os.getenv("SB_USER")
DB_PASSWORD = os.getenv("SB_PASSWORD")
DB_HOST = os.getenv("SB_HOST")
DB_PORT = os.getenv("SB_PORT")
DB_NAME = os.getenv("SB_NAME")

# Get the supabase DB credential for connection
def get_connection():
    return psycopg2.connect(
        user=SB_USER,
        password=SB_PASSWORD,
        host=SB_HOST,
        port=SB_PORT,
        dbname=SB_NAME 
    )

# Get the log table DB credential for connection
def get_connection2():
    return psycopg2.connect(
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME 
    )


In [9]:
def get_users(customers):
    try:
        with get_connection() as conn:
            with conn.cursor() as cur:
                query = """
                    SELECT *
                    FROM phonerequest p
                    WHERE p.emailaddress = ANY(%s);
                """
                cur.execute(query, (customers,))  # Pass as a tuple
                results = cur.fetchall()
        return results
    except Exception as e:
        print(f"Error fetching users: {e}")
        return []

emails = ['jared.wood@example.com', 'terri.hunt@example.com']
users = get_users(emails)
print(users)


[('Jared Wood', 'Jared', '08080862259', 'Data', 'Software Engineer', 'jared.wood@example.com', '475534294', 'New extension including new cabling and socket', 'Smartphone; Mobile phone', 'Temporary use (three months or less)', '2025-10-28', '2025-11-23', 'User requests digital line', datetime.date(2025, 10, 24)), ('Terri Hunt', 'Terri', '09159526326', 'Support', 'IT Support', 'terri.hunt@example.com', '5804338', 'Relocate existing to a new location', 'SIM card only', 'Temporary use (three months or less)', '2025-11-04', '2025-12-03', 'Hot desk setup near meeting rooms', datetime.date(2025, 10, 24))]


In [10]:
#function to retrieve user details (emailaddress)

def retrieve_user_detail(email):
    try:
        with get_connection2() as conn: 
            with conn.cursor() as cur:
                cur.execute("SELECT * FROM phonerequest WHERE emailaddress = %s", (email,))
                a= cur.fetchall()
                print(a)
    except Exception as e:
        print(f'error')


In [11]:
retrieve_user_detail('jared.wood@example.com')

[('Jared Wood', 'Jared', '08080862259', 'Data', 'Software Engineer', 'jared.wood@example.com', '475534294', 'New extension including new cabling and socket', 'Smartphone; Mobile phone', 'Temporary use (three months or less)', '2025-10-28', '2025-11-23', 'User requests digital line', datetime.date(2025, 10, 24))]


In [12]:
# Function to check log table in postgres DB
def is_logged(email):
    try:
        with get_connection2() as conn: 
            with conn.cursor() as cur:
                cur.execute("SELECT 1 FROM integration_ticket_log WHERE email = %s", (email,))
                return cur.fetchone() is not None
    except Exception as e:
        print("Error checking log table:", e)
        return False

In [13]:
# Function to log the ticket status in the log tble

def log_ticket(name, issue_key, email):
    try:
        with get_connection2() as conn:
            with conn.cursor() as cur:
                cur.execute("""
                    INSERT INTO integration_ticket_log
                    (name, issue_key, email, is_ticket_created, is_form_updated)
                    VALUES (%s, %s, %s, 'Yes', 'No')
                    ON CONFLICT (email) DO NOTHING;
                """, (name, issue_key, email))
                conn.commit()
    except Exception as e:
        print("Error logging ticket:", e)

In [ ]:
# Function to create the ticket
def create_jira_ticket(user):
    newusername,samplename,phonenumber,departmentname,job,emailaddress,costcenter,telephonelinesandinstallations,handsetsandheadsets,timeframe,dateneededby,approximateendingdate,Comments,createdat = retrieve_user_detail(user)
    newusername, emailaddress, phonenumber, departmentname, job, costcenter, telephonelinesandinstallations,handsetsandheadsets,timeframe,dateneededby
    payload = {
        "serviceDeskId": SERVICEDESKID,
        "requestTypeId": REQUESTTYPEID,
        "requestFieldValues": {
            "summary": f"New request from {newusername}",
            "description": f"Request submitted by {newusername} ({emailaddress})",
            # Map custom fields here using fieldIds from your second output
            "Name": newusername,
            "Email": emailaddress,
            "Phone": phonenumber,
            "Department": departmentname,
            "Job title": job,
            "Cost center (acct #)": costcenter,
            "Date needed by": dateneededby,
            "Equipment requested": handsetsandheadsets, 
            "Installation requested": telephonelinesandinstallations,
            "Approximateendingdate":approximateendingdate,
            "Comments": Comments,
            "This equipment is for ": timeframe

        }
    }

    response = requests.post(
        f"{JIRA_URL}/rest/servicedeskapi/request",
        headers=headers,
        auth=auth,
        json=payload
    )

    if response.status_code == 201:
        issue_key = response.json().get("issueKey")
        print(f"Ticket created: {issue_key}")
        log_ticket(newusername, issue_key, emailaddress)
    else:
        print(f"Failed to create ticket for {emailaddress}: {response.status_code}, {response.text}")

In [22]:
#create_jira_ticket(retrieve_user_detail('jared.wood@example.com'))
#retrieve_user_detail('jared.wood@example.com')
emails = ['jared.wood@example.com', 'terri.hunt@example.com']
users = retrieve_user_detail(emails)
#field_map = create_jira_ticket()

for user in users:
    create_jira_ticket(users)

error


TypeError: 'NoneType' object is not iterable

In [ ]:
# Main loop

emails_to_process = ["adebola@gmail.com", "jared.wood@example.com"]

users = get_users(emails_to_process)

for user in users:
    if not is_logged(user[1]):
        create_jira_ticket(user)
    else:
        print(f"Skipping {user[1]}: already logged")

Error checking log table: column "email" does not exist
LINE 1: SELECT 1 FROM integration_ticket_log WHERE email = 'Jared'
                                                   ^



ValueError: too many values to unpack (expected 10)

In [19]:
#url = f"{JIRA_URL}/rest/servicedeskapi/servicedesk/{SERVICEDESKID}/requesttype/{REQUESTTYPEID}/field"

url = f'https://api.atlassian.com/jira/forms/cloud/{CLOUD_ID}/servicedesk/{SERVICEDESKID}/requesttype/{REQUESTTYPEID}/form'

#auth = HTTPBasicAuth(admin_email, FIELDVALUE_API)
response = requests.get(url, headers=headers, auth=auth)
response.raise_for_status()
data = response.json()
field_map = {f["name"].strip().lower(): f["fieldId"] for f in data["requestTypeFields"]}
field_map

KeyError: 'requestTypeFields'